In [23]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import NoduleDataset, SubtypedDataLoader


In [26]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Using cpu


# Testing SubtypedDataLoader

In [3]:
lidc_subtyped = pd.read_csv('lidc_subtyped.csv')
lidc_subtyped.head(5)

,Unnamed: 0,Nodule_id,malignancy,subtype
0,0,1197,benign,0benign
1,1,1208,benign,1benign
2,2,1109,benign,1benign
3,3,1259,benign,0benign
4,4,1050,benign,1benign


In [8]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    
    rat = (this_max - this_min)/(this_array.max() - this_array.min())
    this_array = this_array * rat
    this_array -= this_array.min()
    this_array += this_min
    if set_to_int:
        return this_array.to(dtype= torch.int)
    return this_array

In [27]:
def getImages(image_folder):
    '''
        Input:
        image_folder: directory of the image files

        Output:
        m1: list of the labels encountered (1,2,4,5)
        m2: list of binary labels encountered (benign, malignant)
        diff: list of any nodes with discrepency to CSV labels

    '''
    
    M_benign = []
    L_benign = []
    
    M_malignant = []
    L_malignant = []

    

    lidc = pd.read_csv('lidc_subtyped.csv')
    for dir1 in os.listdir(image_folder):
  
        if dir1 == 'Malignancy_3':
            continue

        for file in os.listdir(os.path.join(image_folder, dir1)):
#             malignancy_orig = int(dir1[-1])
#             m1.append(malignancy_orig)

            temp_nodule_ID = file.split('.')[0]
            subtype = lidc[lidc['Nodule_id']==int(temp_nodule_ID)]['subtype'].iloc[0] 
            
            if subtype == '0benign':
                image_array = M_benign
            elif subtype == '1benign':
                image_array = L_benign
            elif subtype == '1malignant':
                image_array = M_malignant
            else:
                image_array = L_malignant
            
            image = np.loadtxt(os.path.join(image_folder, dir1,file))
            image = torch.from_numpy(image).to(DEVICE)
            rgb_image = torch.stack((image,image,image), dim = 0)
            rgb_image = getNormed(rgb_image)
            rgb_image = rgb_image / 255 

            image_array.append(rgb_image)


    return M_benign, L_benign, M_malignant, L_malignant


In [28]:
M_benign, L_benign, M_malignant, L_malignant = getImages('LIDC(MaxSlices)_Nodules')

In [32]:
print(f'''Size of M_benign: {len(M_benign)}, Size of L_benign: {len(L_benign)}, 
Size of M_malignant: {len(M_malignant)}, Size of L_malignant: {len(L_malignant)}''')

Size of M_benign: 519, Size of L_benign: 512, 
Size of M_malignant: 430, Size of L_malignant: 202


In [56]:
subclass_data = {'M_benign': (M_benign, torch.zeros(519)), 
                 'L_benign': (L_benign, torch.zeros(512)),
                 'M_malignant': (M_malignant, torch.ones(430)),
                 'L_malignant': (L_malignant, torch.ones(202))}

In [162]:
a = SubtypedDataLoader(subclass_data, 100)

In [164]:
for idx, b in enumerate(a):
    print(idx)

0
1
2


In [92]:
from itertools import cycle
from torch.utils.data import DataLoader

subclass_Dataset =  NoduleDataset(*subclass_data['L_malignant'])
subclass_iterLoader = cycle(iter(DataLoader(subclass_Dataset, 200, shuffle=True)))

In [158]:
next(a)

StopIteration: 

# Testing DomainBeds dataloader

In [172]:
from fast_data_loader import InfiniteDataLoader

In [165]:
class MultipleDomainDataset:
    N_STEPS = 5001           # Default, subclasses may override
    CHECKPOINT_FREQ = 100    # Default, subclasses may override
    N_WORKERS = 1 #8           # Default, subclasses may override
    ENVIRONMENTS = None      # Subclasses should override
    INPUT_SHAPE = None       # Subclasses should override

    def __getitem__(self, index):
        return self.datasets[index]

    def __len__(self):
        return len(self.datasets)


In [170]:
class ClusterdLIDC(MultipleDomainDataset):
    def __init__(self, root, environments, input_shape,
                 num_classes):
        super().__init__()
        if root is None:
            raise ValueError('Data directory not specified!')
        
        #hard coded: M_benign, L_benign, M_malignant, L_malignant 
        all_images = getImages(root)
        
        self.datasets = []
        
        for i in range(len(environments)):
            images = all_images[i] 
            labels = torch.zeros(len(images)) if i < 2 else torch.ones(len(images))
            self.datasets.append(NoduleDataset(images,labels))

        self.input_shape = input_shape
        self.num_classes = num_classes


In [171]:
datasets = ClusterdLIDC('LIDC(MaxSlices)_Nodules',['M_benign', 'L_benign', 'M_malignant', 'L_malignant'], None, 4)

In [211]:
    train_loaders = [InfiniteDataLoader(
        dataset=env, #each subclass DataSet
        weights=None, #None
        batch_size=200,
        num_workers=1)
        for i, env in enumerate(datasets)]

In [277]:
train_loaders

In [212]:
train_minibatches_iterator = zip(*train_loaders)

In [278]:
train_minibatches_iterator

In [280]:
x = next(train_minibatches_iterator)
x[0][0]

tensor([[[[0.1020, 0.0784, 0.0549,  ..., 0.4431, 0.4510, 0.4314],
          [0.0941, 0.0745, 0.0706,  ..., 0.4510, 0.4275, 0.4314],
          [0.0784, 0.0627, 0.0627,  ..., 0.4549, 0.4471, 0.4549],
          ...,
          [0.1294, 0.1216, 0.0706,  ..., 0.4745, 0.4784, 0.4941],
          [0.1529, 0.1216, 0.0784,  ..., 0.4902, 0.4941, 0.4980],
          [0.1804, 0.1137, 0.0627,  ..., 0.4980, 0.5098, 0.5137]],

         [[0.1020, 0.0784, 0.0549,  ..., 0.4431, 0.4510, 0.4314],
          [0.0941, 0.0745, 0.0706,  ..., 0.4510, 0.4275, 0.4314],
          [0.0784, 0.0627, 0.0627,  ..., 0.4549, 0.4471, 0.4549],
          ...,
          [0.1294, 0.1216, 0.0706,  ..., 0.4745, 0.4784, 0.4941],
          [0.1529, 0.1216, 0.0784,  ..., 0.4902, 0.4941, 0.4980],
          [0.1804, 0.1137, 0.0627,  ..., 0.4980, 0.5098, 0.5137]],

         [[0.1020, 0.0784, 0.0549,  ..., 0.4431, 0.4510, 0.4314],
          [0.0941, 0.0745, 0.0706,  ..., 0.4510, 0.4275, 0.4314],
          [0.0784, 0.0627, 0.0627,  ..., 0